In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
env_name = 'CartPole-v1'

In [3]:
log_path = os.path.join('Training', 'Logs')

In [4]:
env = gym.make(env_name, render_mode="human")
env = DummyVecEnv([lambda: env]) # type: ignore
# ppo_model = PPO('MlpPolicy', env, verbose=1,
#                 tensorboard_log=log_path, n_epochs=10,
#                 learning_rate=0.001)

In [5]:
# ppo_model.learn(total_timesteps=20000)

In [6]:
ppo_model_path = os.path.join('Training', 'Saved Models', 'ppo_model')
# ppo_model.save(ppo_model_path)

In [7]:
# del ppo_model
ppo_model = PPO.load(ppo_model_path, env=env)

In [8]:
# evaluate_policy(ppo_model, env, n_eval_episodes=10, render=True)

In [9]:
def test_model(model):
    for i in range(1):
      obs = env.reset()
      terminated = False
      score = 0
      while not terminated:
          env.render()
          action, _ = model.predict(obs)
          obs, reward, terminated, truncated = env.step(action) # type: ignore
          score += reward # type: ignore
      print('Episode: {}, Score: {}'.format(i, score)) 

In [10]:
# test_model(ppo_model)

Episode: 0, Score: [500.]


In [11]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [14]:
best_model_path = os.path.join('Training', 'Saved Models', 'best_model')

In [15]:
stop_calback = StopTrainingOnRewardThreshold(reward_threshold=250, verbose=0)
eval_callback = EvalCallback(env, callback_on_new_best=stop_calback,
                             eval_freq=10000, verbose=0, best_model_save_path=best_model_path)

In [17]:
new_ppo_model = PPO('MlpPolicy', env, verbose=0, tensorboard_log=log_path)
# new_ppo_model.learn(total_timesteps=20000, callback=eval_callback)

c:\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [19]:
net_arch = dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])
new_ppo_model = PPO('MlpPolicy', env,
                    verbose=0, tensorboard_log=log_path,
                    policy_kwargs={'net_arch': net_arch})

In [20]:
new_ppo_model.learn(total_timesteps=10000, callback=eval_callback)

c:\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [21]:
from stable_baselines3 import DQN

dqn_model = DQN('MlpPolicy', env, verbose=0, tensorboard_log=log_path)
dqn_model.learn(total_timesteps=20000)

In [ ]:
dqn_model_path = os.path.join('Training', 'Saved Models', 'dqn_model')
dqn_model.save(dqn_model_path)

dqn_model = DQN.load(dqn_model_path, env=env)